I'm gonna overwrite a lot of this notebook's old content. I changed the way I'm calculating wt, and wanna test that my training worked. 

In [45]:
from pearce.emulator import *
from pearce.mocks import cat_dict
import numpy as np
from os import path

In [46]:
import matplotlib
#matplotlib.use('Agg')
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

In [47]:
from GPy.models import GPKroneckerGaussianRegression
from GPy.kern import *
import h5py

In [48]:
training_file = '/home/users/swmclau2/scratch/xi_gg_zheng07_cosmo_v3/PearceXiggCosmo.hdf5'
test_file = '/home/users/swmclau2/scratch/xi_gg_zheng07_cosmo_test_v3/PearceXiggCosmoTest.hdf5'

In [49]:
f = h5py.File(training_file, 'r')

In [50]:
Ys = []
rbin = 1
for i in xrange(40):
    Ys.append(f['cosmo_no_%02d'%i]['a_1.000']['obs'].value[:, rbin])

In [51]:
n_hods = 400
start_idx = 0
X1 = f.attrs['cosmo_param_vals']
X2 = f.attrs['hod_param_vals'][start_idx:start_idx+n_hods]
Y = np.vstack(Ys)[:, start_idx:start_idx+n_hods]

In [52]:
X1.shape, X2.shape, Y.shape

((40, 7), (400, 4), (40, 400))

In [53]:
# how to add training errors?

In [54]:
K1 = RBF(input_dim = 7, ARD = False)#+Bias(input_dim=7)# + Linear(input_dim = 7, ARD = True) + Bias(input_dim=7)# + White(input_dim=7)
K2 =  RBF(input_dim=4, ARD = False)#+ Linear(input_dim = 4, ARD = False) + Bias(input_dim=4)# + White(input_dim=4)

In [55]:
model = GPKroneckerGaussianRegression(X1, X2, Y,K1, K2 )

In [56]:
model.optimize_restarts(num_restarts=10)

Optimization restart 1/10, f = -65407.5677227
Optimization restart 2/10, f = -65407.5678369
Optimization restart 3/10, f = -65407.5675335
Optimization restart 4/10, f = -65407.5679218
Optimization restart 5/10, f = -65407.5680814
Optimization restart 6/10, f = -65407.5679854
Optimization restart 7/10, f = -48218.3664352
Optimization restart 8/10, f = -65407.5678236
Optimization restart 9/10, f = -65407.56785
Optimization restart 10/10, f = -65407.5677257


In [57]:
K1.param_array

array([ 9.97263178,  3.25753905])

In [58]:
K2.param_array

array([ 11.18176015,   1.1217286 ])

In [59]:
f2 = h5py.File(test_file, 'r')

In [60]:
Y2s = []
for i in xrange(35):
    Y2s.append(f2['cosmo_no_%02d'%i]['a_1.000']['obs'].value[:, rbin])

In [61]:
testX1 = f2.attrs['cosmo_param_vals']
testX2 = f2.attrs['hod_param_vals']
testY = np.vstack(Y2s)

In [62]:
predY, _ = model.predict(testX1, testX2)

In [63]:
np.median( np.abs( (10**predY[:,0] - 10**testY.flatten(order='F'))/10**testY.flatten(order='F') )  ) 

0.075467499453588827

In [64]:
np.mean( np.abs( (10**predY[:,0] - 10**testY.flatten(order='F'))/10**testY.flatten(order='F') )  ) 

0.093940972449424426

In [65]:
predY[:,0]

array([ 3.64823689,  3.64823689,  3.64823689, ...,  3.5583538 ,
        3.5583538 ,  3.5583538 ])

In [66]:
testY.flatten(order='F')

array([ 3.51843877,  3.52315065,  3.52393741, ...,  3.60349051,
        3.60667587,  3.60662163])